In [1]:
%pwd

'/Users/kangb3/github/project_spring_2020'

In [2]:
from class_setup import create_package_dir
create_package_dir("bk_packages")

Starting in /Users/kangb3/github/project_spring_2020
Creating /Users/kangb3/github/project_spring_2020/bk_packages...
The current working directory is now /Users/kangb3/github/project_spring_2020/bk_packages


In [4]:
%cd ..

/Users/kangb3/github/project_spring_2020


In [6]:
from pathlib import Path
package_name = "bk_packages"
#python_dir = Path(package_name)
#python_dir.mkdir()
(python_dir / '__init__.py').touch()
Path('setup.py').touch()
Path('LICENSE').touch()
Path('README.md').touch()
Path('tests').mkdir()

In [7]:
%%writefile setup.py
import setuptools

with open("README.md", "r") as fh:
    long_description = fh.read()

setuptools.setup(
    name="bk_packages", 
    version="0.0.1",
    author="byunghyun_kang",
    author_email="danjong99@gmail.com",
    description="A small python package which downsizes input scRNAseq matrix",
    long_description=long_description,
    long_description_content_type="text/markdown",
    url="https://github.com/danjong99/project_spring_2020",
    packages=setuptools.find_packages(),
    classifiers=[
        "Programming Language :: Python :: 3",
        "License :: OSI Approved :: MIT License",
        "Operating System :: OS Independent",
    ],
    python_requires='>=3.6',

)

Overwriting setup.py


'/Users/kangb3/github/project_spring_2020'

In [2]:
%%writefile README.md
# Single Cell Random Drawing

This python package will enable for users to reduce the size of input scRNAseq matrix by random selection of a given number of samples within pre-defined clusters and averaging the gene expression value of the chosen cells.

# Import
  - from bk_packages.shrink_mtx import rand_draw

# Input
  1. Expression matrix of scRNAseq (gene by cells) - mtx
  2. Metadata (cell ids and cluster or cell type info) - meta
  3. For visualization, pre-calculated tSNE or UMAP coordinates can be tossed along with metadata

# Description

  1. Using well-built packages e.g. numpy and pandas are avoided as much as possible because this is class final project.
     However, numpy, pandas, matplotlib and seaborn used for visualization of input and output.
  2. I used class, which seems redundant in this case, for practice. Version w/o class is in old code folder (but not complete).
  3. How it works.
     'rand_draw' class has 4 essential methods (annotated by Alphabet A-D) to accomplish the intended task.
        - First, generate an object (obj = rand_draw(mtx, meta, num_to_select))
        - you should perform obj.A-(), .B-(), .C-() and .D-() in order.
        - obj.E-() is brief report of the result.
        - obj.tSNE_intput() / .tSNE_output() give original/shrunken tSNE plot, repectively.
    
# Returns
  1. shrunken matrix
  2. shrunken metadata
  3. cell.ids of randomly selected cells
  4. tSNE plot, if the coordinates were given.

# The benefit of downsizing cells with averaging the exprssion levels of gene?
  1. downsizing itself reduce the calculation burden --> reducing calculation time
  2. Increase the resolution of differential gene expressin.

(e.g. assume that you have 400 cells in cluster 1 of your scRNAseq data, and if you set the number of cells to be chosen as 20, then 20 cells from cluster 1 of your data will be randomly and repeatedly selected with no replacement. This process will downsize the cluster 1 from 400 cells to 20 cells with better gene expression resolution.)

Overwriting README.md


In [9]:
%%writefile LICENSE
Copyright (c) 2018 The Python Packaging Authority

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

Overwriting LICENSE


In [27]:
%pwd

/Users/kangb3/github/project_spring_2020


In [28]:
%%writefile bk_packages/basic_methods.py

def unique(input_list):
    """Return unique value of the input list"""  
    try:
        # intilize a null list 
        unique_list = [] 
        # traverse for all elements 
        for x in input_list: 
            # check if exists in unique_list or not 
            if x not in unique_list: 
                unique_list.append(x)
        return(unique_list)
    except TypeError as detail:
        return ("int object is not iterable")

def t(mtx):
    """T-transform the Input Matrix"""
    try:
        if type(mtx) is str: #if the input is str, raise exception.
            raise Exception('Error: the input should be a list of lists.')
        
        for i in range(0, len(mtx)):
            assert len(mtx[i]) == len(mtx[i-1]) # check whether each list has the same length.
        
        ## Execution code
        mtx_t = []
        for j in range(0, len(mtx[0])):
            mtx_t.append( [mtx[i][j] for i in range(0,len(mtx))] )
        return(mtx_t)
    
    except TypeError as detail: # Control TypeEorror.
        return ('Error: the input should be a list of lists.')
    except AssertionError as detail:
        return ('Error: length of the lists should be the same')

def convert_to_float(input_list):
    """Convert list of str to list of float"""
    try:
        list_float = [float(i) for i in input_list]
        return(list_float)
    except ValueError as detail:
        return ("input is not convertible to float.")

def rowSum(mtx):
    """Return all row-sums as a list"""
    try:
        for i in range(0, len(mtx)):
            assert len(mtx[i]) == len(mtx[i-1]) # check whether each list has the same length.
        
        res = list()
        for j in range(0, len(mtx[0])): 
            tmp = 0
            for i in range(0, len(mtx)): 
                tmp = tmp + mtx[i][j]
            res.append(tmp)
        return(res)
    
    except AssertionError as detail:
        return ('Length of lists is irregular or input format is wrong.')
    except TypeError as detail:
        return ('Undefined operand type')

def rowMean(mtx):
    """Return all row-sums as a list"""
    try:
        for i in range(0, len(mtx)):
            assert len(mtx[i]) == len(mtx[i-1]) # check whether each list has the same length.
        
        res = list()
        for j in range(0, len(mtx[0])): 
            tmp = 0
            for i in range(0, len(mtx)): 
                tmp = tmp + mtx[i][j]
            res.append(tmp/len(mtx))
        return(res)
    
    except AssertionError as detail:
        return ('Length of lists is irregular or input format is wrong.')
    except TypeError as detail:
        return ('Undefined operand type')

Overwriting bk_packages/basic_methods.py


In [44]:
%%writefile bk_packages/shrink_mtx.py

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from bk_packages.basic_methods import *
import random
import math
from bk_packages.basic_methods import unique, t, convert_to_float, rowMean

class rand_draw():
    """A simple way to visualize the results"""

    def __init__(self, mtx, meta, num_to_select=20):
        
        """Initialize object's attributes"""
        
        self.mtx = mtx
        self.meta = meta
        self.num_to_select = num_to_select
        
        self.meta_by_cluster = []
        self.meta_dic = {}
        self.mtx_dic = {}
        self.dic_random_items = {}
        self.mtx_small = {}
        
    def A_clustocell_dic_generator(self):
        
        """Return {cluster:cell.ids} dictionary """
        
        temp = [self.meta[i][2] for i in range(0,len(self.meta))] # extract cell cluster info
        unique_cluster = unique(temp[1:])         # take unique cluster info
        unique_cluster = sorted(unique_cluster)

        ## group cell.ids based on their cluster
        for cluster in unique_cluster:
            index = [i for i, n in enumerate(temp) if n == cluster] # extract the indices of rows in a given cluster
            a = [self.meta[i][1:3] for i in index] # using the indices, extract cell.id and assigned cluster info.
            self.meta_by_cluster.append(a)
    
        # convert the list acquired above to dictionary with the cluster as keys
        for j in range(0, len(unique_cluster)):
            self.meta_dic[ self.meta_by_cluster[j][0][1] ] = [ self.meta_by_cluster[j][i][0] for i in range(0, len(self.meta_by_cluster[j]))]
    
    def B_celltogene_dic_generator(self):
        """Apply T-transformation and convert the list of lists to Dictionary"""
        """with cell.id as keys and gene expression as value"""
        mtx_t = t(self.mtx)
        for line in mtx_t:
            self.mtx_dic[line[0]] = line[1:]
    
    def C_random_selector(self, num_to_select = None):
        """Random selection of cell.ids from each cluster"""
            
        if num_to_select != None:
            if isinstance(num_to_select, int):
                self.num_to_select = num_to_select
            else:
                raise Exception('Error: "num_to_select" undefined type')
                return 0
        
        self.dic_random_items = {} # reset dic_random_items
        for key in self.meta_dic.keys():
                        
            clust_cell_ids = self.meta_dic[key].copy()

            # determine the number of cycle to run in the following loop.
            if len(self.meta_dic[key]) < self.num_to_select:
                print("Number of items to select exceeds the number of items in the list.")
                print("Remember this is random selection without replacement.")
                cycle_to_run = math.floor(len(self.meta_dic[key])/self.num_to_select)
            elif len(self.meta_dic[key])%self.num_to_select == 0:
                cycle_to_run = int(len(self.meta_dic[key])/self.num_to_select)
            else:
                cycle_to_run = math.ceil(len(self.meta_dic[key])/self.num_to_select)

            for i in range(0, cycle_to_run): # random sampling from the given list without replacement
                if len(clust_cell_ids) > self.num_to_select:
                    a = random.sample(clust_cell_ids, self.num_to_select)
                    self.dic_random_items[ a[0] ] = a  # takes the first cell.id among selected ones as a key.
                    [clust_cell_ids.remove(a[i]) for i in range(0,len(a))] # selected cell.ids are removed from the dic.
                elif len(clust_cell_ids) < self.num_to_select: # the remaining cells are taken 
                    a = clust_cell_ids
                    self.dic_random_items[ a[0] ] = a
    
    def D_avg_gene(self):
        
        """Average gene expression values by defined rowMean fuction"""
        
        a = self.mtx_dic['gene'].copy()
        self.mtx_small['gene'] = a
    
        for key, values in self.dic_random_items.items():
            temp = []
            a = []
            for value in values:
                a = self.mtx_dic[ value ].copy()
                temp.append(convert_to_float(a))
            self.mtx_small[ key ] = rowMean(temp)
    
        #[v.insert(0,k) for k, v in mtx_small.items()]
        #temp_2 = [x for x in mtx_small.values()]    
        #temp = t(temp_2)
        #self.final_mtx = temp.copy()
    
    def E_briefing(self):
        
        """Briefly reports the result of random selection"""
        
        meta_by_clus_temp = []
        for i in range(0, len(self.meta_by_cluster)):
            for j in range(0, len(self.meta_by_cluster[i])):
                meta_by_clus_temp.append(self.meta_by_cluster[i][j])
        
        meta_by_clus_dic = {}
        for i in range(0, len(meta_by_clus_temp)):
            meta_by_clus_dic[ meta_by_clus_temp[i][0] ] = meta_by_clus_temp[i][1]

        for k, v in self.dic_random_items.items():
            print( "Key Cell ID: " + k + " - Assigned Cluster: " + str(meta_by_clus_dic[k]) + " - Number of Cell: " + str(len(v)))

        
    def tSNE_input(self):
        
        """Visualize tSNE cooridinates from original metadata"""
               
        temp = pd.DataFrame.from_dict(self.meta)
        temp.columns = [temp.iloc[0][i] for i in range(0,len(temp.columns))]
        pd_meta = temp.iloc[1:].copy()
        pd_meta[['tSNE_1','tSNE_2']] = pd_meta[['tSNE_1','tSNE_2']].astype(float)

        sns.lmplot(x='tSNE_1',
                   y='tSNE_2',
                   data=pd_meta,
                   fit_reg=False,
                   legend=True,
                   height=9,
                   hue='cluster',
                   scatter_kws={"s":200, "alpha":0.3})

        title = ('tSNE Plot of %s Mouse Colon Cells'% len(pd_meta))
        plt.title(title, weight='bold').set_fontsize('14')
        plt.xlabel('tSNE 1', weight='bold').set_fontsize('20')
        plt.ylabel('tSEN 2', weight='bold').set_fontsize('20')
    
    
    def tSNE_output(self):
        
        """Visualize tSNE cooridinates from shrunken metadata"""
        
        if bool(self.mtx_small):
            
            temp = pd.DataFrame.from_dict(self.meta)
            temp.columns = [temp.iloc[0][i] for i in range(0,len(temp.columns))]
            pd_meta = temp.iloc[1:].copy()

            pd_mtx_small = pd.DataFrame.from_dict(self.mtx_small)
            pd_meta.index = pd_meta['cell.id']
            df = pd_meta[ pd_meta.index.isin(pd_mtx_small.columns) ]       
            df[['tSNE_1','tSNE_2']] = df[['tSNE_1','tSNE_2']].astype(float)

            sns.lmplot(x='tSNE_1',
                       y='tSNE_2',
                       data=df,
                       fit_reg=False,
                       legend=True,
                       height=9,
                       hue='cluster',
                       scatter_kws={"s":200, "alpha":0.3})

            title = ('tSNE Plot of %s Mouse Colon Cells'% len(df))
            plt.title(title, weight='bold').set_fontsize('14')
            plt.xlabel('tSNE 1', weight='bold').set_fontsize('20')
            plt.ylabel('tSEN 2', weight='bold').set_fontsize('20')
        
        else:
            print("shrunken mtx is empy. A,B,C methods should be run.")

Overwriting bk_packages/shrink_mtx.py


In [45]:
%%writefile tests/test_basics.py

from bk_packages.basic_methods import t, convert_to_float, unique, rowSum, rowMean

def test_t():
    input_list = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
    obs = t(input_list)
    exp = [[1, 4, 7, 10], [2, 5, 8, 11], [3, 6, 9, 12]]
    assert obs == exp

def test_convertTofloat():
    input_list = ['1','2','3','4','5','6','7','8','9','10']
    obs = convert_to_float(input_list)
    exp = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
    assert obs == exp

def test_unique():
    input_list = [1,1,1,2,2,2,3,3,4,4,5,5,6,6,7,8,9,9,9,9,9,9,10]
    obs = unique(input_list)
    exp = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    assert obs == exp

def test_rowSum():
    input_list = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
    obs = rowSum(input_list)
    exp = [22, 26, 30]
    assert obs == exp

def test_rowMean():
    input_list = [[1,2,3],[4,5,6],[7,8,9],[10,11,12]]
    obs = rowMean(input_list)
    exp = [5.5, 6.5, 7.5]
    assert obs == exp

Overwriting tests/test_basics.py


In [14]:
%%writefile tests/test_shrink_mtx.py

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from bk_packages.shrink_mtx import rand_draw

#### Read Meta Files - read as list of lists
meta = []
with open("tests/metatable.txt") as f_obj:
     for line in f_obj:
        res = line.replace('"','').strip('\n').split('\t')
        meta.append(res)
    
### Insert one element in the 1th list of the meta data
meta[0].insert(0,'rowname')
    
### Read matrix file - read as list of lists
mtx = []
with open("tests/exprs_mtx.txt") as f_obj:
    for line in f_obj:
        res = line.replace('"','').strip('\n').split('\t')
        mtx.append(res)
    
### Insert one element in the 1th list of the mtx
mtx[0].insert(0,'gene')

my_data = rand_draw(mtx=mtx, meta=meta, num_to_select=20)

def test_clustocell_dic_generator():
    my_data.A_clustocell_dic_generator()
    obs = []
    for i in range(0, len(my_data.meta_by_cluster)):
        obs.append(len(my_data.meta_by_cluster[i]))
    exp = [200,200,200,132,57,200,200,200,200,200,200,200,200]
    assert obs == exp

def test_clustocell_dic_generator():
    my_data.A_clustocell_dic_generator()
    obs = {}
    for k, v in my_data.meta_dic.items():
        obs[k] = len(v)
    exp = {'0': 200,'1': 200,'10': 200,'11': 132,'12': 57,'2': 200,'3': 200,'4': 200,'5': 200,'6': 200,'7': 200,'8': 200,'9': 200}
    assert obs == exp

def test_celltogene_dic_generator():
    my_data.B_celltogene_dic_generator()
    a = []
    for k in my_data.mtx_dic.keys():
        a.append(k)
    obs = {}
    obs[a[0]] = len(a[1:])
    exp = {'gene': 2389}
    assert obs == exp

def test_random_selector():
    my_data.C_random_selector(num_to_select=56)
    obs = []
    for k, v in my_data.dic_random_items.items():
        obs.append(len(v))
    exp = [56,56,56,32,56,56,56,32,56,56,56,32,56,56,20,56,1,56,56,56,32,56,56,56,
           32,56,56,56,32,56,56,56,32,56,56,56,32,56,56,56,32,56,56,56,32,56,56,56,32]
    assert obs == exp

Overwriting tests/test_shrink_mtx.py


In [5]:
%%writefile bk_packages/main.py

def main():
    
if __name__ == '__main__':
    main()

Overwriting bk_packages/main.py


In [15]:
!pip install -e .

Obtaining file:///Users/kangb3/github/project_spring_2020
  Attempting uninstall: bk-packages
    Found existing installation: bk-packages 0.0.1
    Uninstalling bk-packages-0.0.1:
      Successfully uninstalled bk-packages-0.0.1
  Running setup.py develop for bk-packages


In [16]:
!pytest

============================= test session starts ==============================
platform darwin -- Python 3.7.1, pytest-4.1.0, py-1.7.0, pluggy-0.8.0
rootdir: /Users/kangb3/github/project_spring_2020, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.1, doctestplus-0.2.0, dependency-0.4.0, arraydiff-0.2
collected 8 items                                                              

tests/test_basics.py .....                                               [ 62%]
tests/test_shrink_mtx.py ...                                             [100%]

=============================== warnings summary ===============================
/anaconda3/lib/python3.7/site-packages/pandas/core/dtypes/inference.py:6
  /anaconda3/lib/python3.7/site-packages/pandas/core/dtypes/inference.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
    from collections import Iterable

/anaconda3/lib/python3.7/site-packag